# Автоматическая обработка текстов
## Домашнее задание 1 [10 баллов] до 23:59 15.03.2018
**Студент: Андрей Саутин**  
**Группа: 396**

В этом домашнем задании требуется написать генератор описания прогноза погоды на следующую неделю в каком-нибудь городе. Домашнее задание состоит из трех частей:
1. Скачивание данных о состоянии погоды в городе 
2. Генерация описания прогноза
3. Творческая часть


### 1. Сбор данных [3 балла]

Воспользуемся бесплатным API сервиса [Apixu](https://www.apixu.com/), который также предоставляет библиотеку [apixu-python](https://github.com/apixu/apixu-python) для работы с ним. Подгрузим прогноз погоды на 7 дней для города `LOCATION`.

In [1]:
from apixu.client import ApixuClient
from os import environ
import locale

API_KEY = environ['APIXU_KEY']
LOCATION = 'Рига'
DATE_FORMAT = '%d.%m (%a)'
DAY_COUNT = 7

locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')
client = ApixuClient(API_KEY)
forecast = client.getForecastWeather(q=LOCATION, days=DAY_COUNT)['forecast']['forecastday']

Создадим обертку `DateWrapper` над встроенным классом `datetime` для более удобной работы с ним в рамках текущего задания.

In [2]:
class DateTimeWrapper:
    def __init__(self, date, repr_format=None, str_format=None):
        self._date = date
        self._repr_format = repr_format
        self._str_format = str_format 

    def __repr__(self):
        return self._date.__repr__() if self._repr_format is None else self._date.strftime(self._repr_format)

    def __str__(self):
        return self._date.__str__() if self._str_format is None else self._date.strftime(self._str_format)
    
    def get_date(self):
        return self._date
    
    def get_weekday(self):
        return self._date.strftime('%A').lower()

Оформим весь загруженный прогноз в виде таблицы. В этом мне поможет класс pandas.DataFrame, созданный для удобной работы с данными. В таблице каждый столбец соответствует некоторой погодной характеристике, а каждая строка — дате.

In [3]:
import pandas as pd
from datetime import datetime


aspects = ['минимальная температура', 'максимальная температура',
           'скорость ветра', 'уровень осадков']
attrs = ['mintemp_c', 'maxtemp_c', 'maxwind_kph', 'totalprecip_mm']

dates = [DateTimeWrapper(datetime.strptime(day['date'], '%Y-%m-%d'),
                         repr_format=DATE_FORMAT,
                         str_format=DATE_FORMAT)
         for day in forecast]
d = {aspect_name: [int(round(weather['day'][aspect_attr])) for weather in forecast]
     for aspect_name, aspect_attr in zip(aspects[:-1], attrs[:-1])}
d[aspects[-1]] = [weather['day'][attrs[-1]] for weather in forecast]
dataframe = pd.DataFrame(d, index=dates, columns=aspects)
print(dataframe)

            минимальная температура  максимальная температура  скорость ветра  \
15.03 (Чт)                       -6                        -3              23   
16.03 (Пт)                       -6                        -3              24   
17.03 (Сб)                       -4                        -2              22   
18.03 (Вс)                       -2                         1              13   
19.03 (Пн)                       -1                         2              15   
20.03 (Вт)                       -4                         0              26   
21.03 (Ср)                       -3                        -3              27   

            уровень осадков  
15.03 (Чт)              0.0  
16.03 (Пт)              0.4  
17.03 (Сб)              0.0  
18.03 (Вс)              0.0  
19.03 (Пн)              0.0  
20.03 (Вт)              4.7  
21.03 (Ср)              0.5  


### 2. Генератор описания прогноза погоды [4 балла]

Прогноз погоды должен состоять из нескольких предложений, генерируемых по шаблонам.

Шаблоны, используемые в этом задании:  

1) Изменение температуры:  
**<span style="color:indigo">В {город} [к|ко] {день недели} {глагольная конструкция} в среднем на {количество градусов} [градус|градуса|градусов] относительно {день недели}.</span>**  
Здесь **{глагольная конструкция} = [потеплеет|температура повысится|температура увеличится|температура поднимется|температура возрастет|похолодает|температура понизится|температура уменьшится|температура опустится|температура упадет]**.  

Если температура изменилась незначительно или не изменилась вовсе, то используется немного видоизмененный шаблон:  
**<span style="color:indigo">В {город} [к|ко] {день недели} температура в среднем [не изменится|не поменяется|изменится незначительно|останется прежней].</span>**  

2) Скорость ветра:  
**<span style="color:indigo">Скорость ветра {глагольная конструкция} и составит {скорость} [километр|километра|километров] в час.</span>**  
Здесь **{глагольная конструкция} = [повысится|увеличится|поднимется|возрастет|понизится|уменьшится|опустится|упадет|не изменится|не поменяется|изменится незначительно|останется прежней]**.  

3) Осадки:  
**<span style="color:indigo">За [ближайший|ближайшие] {количество дней} [день|дня|дней] выпадет {количество осадков} миллиметра осадков, а [в|во] {день недели} [— {количество осадков} миллиметра|осадков не ожидается].</span>**

Здесь после количества осадков используется форма "миллиметра", т.к. существительным в такой ситуации управляет дробь (см. [gramota, вопрос № 268857](http://new.gramota.ru/spravka/buro/search-answer?s=%D0%B4%D1%80%D0%BE%D0%B1%D1%8C)).

Для генерации описания прогноза создам класс `WeatherReporter` с публичными методами `compare_temperature`, `compare_wind` и `calc_precipitation` для генерации предложений по шаблонам 1, 2 и 3 соответственно. Также заведу метод `make_report`, который сгенерирует по предложению на каждый шаблон и вернет мне текст описания прогноза.

Ввиду ограничений библиотеки pymorphy2 (или моего с ней недостаточного знакомства), некоторые (нежесткие) части предложений придется сгенерировать вручную, например, согласовать предлоги "к/ко" и "в/во" с днями недели. Для внесения некоторого разнообразия в описание прогноза в процессе его формирования используется генератор случайных чисел для выбора случайного слова из набора синонимичных глаголов.

In [4]:
import random
import pymorphy2


morph = pymorphy2.MorphAnalyzer()


class WeatherReporter:
    def __init__(self, dataframe, location):
        self._dataframe = dataframe
        self._location = location
        self._verbs_go_up = ['повысится', 'увеличится', 'поднимется', 'возрастет']
        self._verbs_go_down = ['понизится', 'уменьшится', 'опустится', 'упадет']
        self._verbs_no_changes = ['не изменится', 'не поменяется', 'изменится незначительно', 'останется прежней']
    
    @staticmethod
    def _inflect_form(word, grammemes):
        return morph.parse(word)[0].inflect(grammemes).word
    
    @staticmethod
    def _make_agree_with_number(word, number):
        return morph.parse(word)[0].make_agree_with_number(number).word
    
    @staticmethod
    def _join_sentence(parts):
        return ' '.join(parts) + '.'
    
    def _make_temperature_sentence(self, date_x, date_y, temp_x, temp_y):
        verbs_warm = ['потеплеет', *map(lambda verb: 'температура ' + verb, self._verbs_go_up)]
        verbs_cool = ['похолодает', *map(lambda verb: 'температура ' + verb, self._verbs_go_down)]
        parts = ['В',
                 self._inflect_form(self._location, {'sing', 'loct'}).title(),
                 'к' if date_y.get_date().isoweekday() != 2 else 'ко',
                 self._inflect_form(date_y.get_weekday(), {'sing', 'datv'})]
        temp_diff = temp_y - temp_x
        temp_diff = int(round(abs(temp_diff)) * (-1 if temp_diff < 0 else 1))
        if temp_diff == 0:
            parts.extend(['температура в среднем',
                          random.choice(self._verbs_no_changes)])
        else:
            parts.append(random.choice(verbs_cool if temp_diff < 0 else verbs_warm))
            parts.extend(['в среднем на',
                          str(abs(temp_diff)),
                          self._make_agree_with_number('градус', abs(temp_diff)),
                          'относительно',
                          self._inflect_form(date_x.get_weekday(), {'sing', 'gent'})])
        return self._join_sentence(parts)

    def _make_wind_sentence(self, speed_x, speed_y):
        parts = ['Скорость ветра']
        speed_diff = speed_y - speed_x
        if speed_diff == 0:
            parts.append(random.choice(self._verbs_no_changes))
        else:
            parts.append(random.choice(self._verbs_go_down if speed_diff < 0 else self._verbs_go_up))
        parts.extend(['и составит',
                      str(speed_y),
                      self._make_agree_with_number('километр', speed_y),
                      'в час'])
        return self._join_sentence(parts)
    
    def _make_precipitation_sentence(self, day, precipitation, precipitation_total):
        date = self._dataframe.index.values[day]
        parts = ['За',
                 self._inflect_form('ближайший', {'masc', 'sing', 'nomn'} if day <= 1 else {'plur', 'nomn'}),
                 str(day),
                 self._make_agree_with_number('день', day),
                 'выпадет',
                 str('{0:.2f}'.format(precipitation_total)),
                 'миллиметра',
                 'осадков, а',
                 'в' if date.get_date().isoweekday() != 2 else 'во',
                 self._inflect_form(date.get_weekday(), {'sing', 'accs'})]
        if precipitation == 0:
            parts.append('осадков не ожидается')
        else:
            parts.extend(['—',
                          str('{0:.2f}'.format(precipitation)),
                          'миллиметра'])
        return self._join_sentence(parts)

    def compare_temperature(self, day_x, day_y):
        dates = self._dataframe.index.values
        temp_x_average = self._dataframe.iloc[day_x, [0, 1]].mean()
        temp_y_average = self._dataframe.iloc[day_y, [0, 1]].mean()
        return self._make_temperature_sentence(dates[day_x], dates[day_y], temp_x_average, temp_y_average)

    def compare_wind(self, day_x, day_y):
        dates = self._dataframe.index.values
        wind_x, wind_y = self._dataframe.iloc[day_x, 2], self._dataframe.iloc[day_y, 2]
        return self._make_wind_sentence(wind_x, wind_y)
    
    def calc_precipitation(self, day):
        precipitation_total = self._dataframe.iloc[:day + 1, 3].sum()
        precipitation = self._dataframe.iloc[day, 3]
        return self._make_precipitation_sentence(day, precipitation, precipitation_total)
    
    def make_report(self, day_x, day_y):
        return ' '.join([
            self.compare_temperature(day_x, day_y),
            self.compare_wind(day_x, day_y),
            self.calc_precipitation(day_y)
        ])

Генерирую два дня, соблюдая условие, что первый день должен идти раньше второго, а затем прошу класс `WeatherReporter` построить описание прогноза. При повторном запуске получаю прогноз для другой пары дней, написанный также немного другими словами.

In [5]:
reporter = WeatherReporter(dataframe, LOCATION)
day_x = random.randint(0, DAY_COUNT - 2)
day_y = random.randint(day_x + 1, DAY_COUNT - 1)
print(reporter.make_report(day_x, day_y))

В Риге к среде температура упадет в среднем на 4 градуса относительно понедельника. Скорость ветра возрастет и составит 27 километров в час. За ближайшие 6 дней выпадет 5.60 миллиметра осадков, а в среду — 0.50 миллиметра.


### 3. Ответьте на вопросы [3 балла]
**1. В каких других задачах (помимо описания прогноза погоды) может понадобиться генерировать текст по шаблонам? В каких задачах может понадобиться генерировать текст об изменении числовых показателей по шаблонам?**

Генерация текста с числовыми показателями по шаблонам может быть полезна в большом количестве задач, связанных с подсчетом статистик, характерных для конкретных сервисов. Вот примеры некоторых из них:  

* составление отчета по изменениям курсов на валютной бирже: *"Японская иена выросла за прошедшую неделю."* или *"Курс доллара США опустился с ... до ..."*;
* отображение статистики посещения сайта: *"36551 уникальное посещение"* или *"222 пользователя сейчас онлайн"*;
* ведение системы контроля учета версий: *"512 коммитов, 8 веток, 3 релиза, 20 коллабораторов"* (а-ля GitHub);
* оповещения социальной сети, блога или электронной почты: *"У вас 11 друзей"* или *"С прошлого года вы сделали 1234 поста"*, или *"У вас 2 непрочитанных сообщения/письма"*;
* составление писем для автоматической рассылки: *"Ваша подписка заканчивается через 3 дня"*, *"Стоимость покупки: 522 рубля"*, *"За прошедшую неделю вы прошли 2 урока и сделали 5 заданий"* (Stepik);
* интернет-магазины: *"Мы подобрали для Вас 115 товаров"*, *"Заказ можно будет забрать в одном из 189 пунктов самовывоза"*, *"Стоимость доставки: 300 руб."* (здесь подразумевается, что стоимость может варьироваться в зависимости от характеристик заказа: его стоимости, общего веса).  

**2. Шаблоны, которые вы использовали в этом задании, имеют фиксированную структуру. Фактически, ваша задача заключалась в том, чтобы подставить в шаблон число и согласовать единицы измерения с этим числом или подставить в шаблон название города и согласовать его с предлогом. Как можно разнообразить эти шаблоны? Как знание синтаксической структуры предложения может помочь в этой задаче?**  

Чтобы разнообразить шаблоны, можно сделать их более гибкими, например, при отсутствии изменения какой-либо статистики генерировать текст, отличный от базового шаблона (вместо "температура изменилась на 0 градусов" можно писать "температура не изменилась"). В этом задании в случае незначительного изменения температуры я выводил текст, отличный от основного шаблона. Таким образом, при подстановке числа или слова в шаблон, уже будет требоваться не просто согласовать грамматические формы слов, но и проверить несколько условий.

Также при составлении более длинных текстов по шаблонам могло бы понадобиться сохранять информацию о значении, подставленном в одном предложении, и проносить ее дальше по тексту. Например, можно представить, что мы хотим сгенерировать сказку, в которой персонажей может выбирать пользователь (ранее в рунете составление таких сказок/рассказов было популярным развлечением), тогда персонаж "Алиса" будет требовать согласования с женским родом во всем тексте, персонаж "Боб" — с мужским родом, а "Привидение" — со средним (что проявится и в частых местоимениях: он/она, его/ее, ей/ему и т.д.).
    
Опять-таки в информации о заказе, присылаемой нам от интернет-магазинов, можно встретить примеры более сложных шаблонов, когда на место плейсхолдера подставляется не одно слово в инфинитиве, а несколько, каждое из которых требует соблюдения своих правил согласования:
*"Вы заказали {количество} {товар}."*, например, "Вы заказали 2 стиральных машины" (с числительным согласуется слово существительное, а прилагательное согласуется с существительным) или "Вы заказали 5 столов для игры в теннис" (с числительным согласуется только слово "стол").